In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk
import torch

In [2]:
horos = pd.read_csv('horoscopes_SA.csv', 
                         sep=';',
                         on_bad_lines='skip',
                         names=["ID", "ROBERTA", "DATE", "SIGN", "VADERS"])
horos = horos. tail(-1)
print(horos.shape)
print(horos.head(3))

(12946, 5)
    ID               ROBERTA        DATE   SIGN  VADERS
1  0.0  -0.22425956125614413  12-01-2013  aries  0.7102
2  1.0  -0.25031325221061707  12-02-2013  aries    0.34
3  2.0    0.7164381593465805  12-03-2013  aries  0.4939


In [3]:
stocks = pd.read_csv('historical_stock_prices.csv', 
                         sep=',',
                         on_bad_lines='skip',
                         names=["TICKER", "OPEN", "CLOSE", "ADJ_CLOSE", "LOW", "HIGH", "VOLUME", "DATE"])
stocks = stocks.tail(-1)
print(stocks.shape)
print(stocks.head(3))

C:\Users\m1273747\AppData\Local\Temp\ipykernel_15996\3650064621.py:1: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  stocks = pd.read_csv('historical_stock_prices.csv',


(20973889, 8)
  TICKER              OPEN             CLOSE         ADJ_CLOSE    LOW  \
1    AHH              11.5  11.5799999237061  8.49315452575684  11.25   
2    AHH  11.6599998474121  11.5500001907349  8.47115135192871   11.5   
3    AHH  11.5500001907349  11.6000003814697  8.50782203674316   11.5   

               HIGH   VOLUME        DATE  
1  11.6800003051758  4633900  2013-05-08  
2  11.6599998474121   275800  2013-05-09  
3  11.6000003814697   277100  2013-05-10  


In [4]:
tickers = stocks.groupby("TICKER").DATE.agg(["min"]).rename(columns={"min": "BD"}) 
print(tickers)

                BD
TICKER            
A       1999-11-18
AA      1970-01-02
AABA    1996-04-12
AAC     2018-01-16
AAL     2005-09-27
...            ...
ZTS     2013-02-01
ZUMZ    2005-05-06
ZUO     2018-04-12
ZYME    2017-04-28
ZYNE    2015-08-05

[5685 rows x 1 columns]


In [5]:
horos['DATE'] = pd.to_datetime(horos.DATE).dt.strftime('%Y-%m-%d')
print(horos)

            ID                ROBERTA        DATE    SIGN   VADERS
1          0.0   -0.22425956125614413  2013-12-01   aries   0.7102
2          1.0   -0.25031325221061707  2013-12-02   aries     0.34
3          2.0     0.7164381593465805  2013-12-03   aries   0.4939
4          3.0    0.17077109217643738  2013-12-04   aries   0.9581
5          4.0     0.9309244230389595  2013-12-05   aries   0.8268
...        ...                    ...         ...     ...      ...
12942  12941.0     0.3253805801726079  2016-12-08  pisces   0.2876
12943  12942.0     0.5431529823690653  2016-12-09  pisces   0.8176
12944  12943.0     0.3677939251065254  2016-12-10  pisces   0.3612
12945  12944.0  -0.026866686121620402  2016-12-11  pisces  -0.2842
12946  12945.0     0.8509626428131014  2016-12-12  pisces   0.8839

[12946 rows x 5 columns]


In [25]:
print("min: ", min(horos['DATE']))
print("max: ", max(horos['DATE']))

print("min: ", min(stocks['DATE']))
print("max: ", max(stocks['DATE']))

min:  2013-12-01
max:  2016-12-12
min:  2013-12-02
max:  2016-12-12


In [6]:
minhoros = min(horos['DATE'])
maxhoros = max(horos['DATE'])
stocks = stocks[(stocks['DATE'] >= minhoros) & (stocks['DATE'] <= maxhoros)]

In [7]:
def zodiac_sign(day, month):
    if (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return "capricorn"
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return "aquarius"
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return "pisces"
    elif (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return "aries"
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return "taurus"
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return "gemini"
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return "cancer"
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return "leo"
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return "virgo"
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return "libra"
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return "scorpio"
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return "sagittarius"

In [8]:
tickers['BD'] = pd.to_datetime(tickers['BD'])
tickers['SIGN'] = tickers['BD'].apply(lambda x: zodiac_sign(x.day, x.month))
print(tickers)

               BD       SIGN
TICKER                      
A      1999-11-18    scorpio
AA     1970-01-02  capricorn
AABA   1996-04-12      aries
AAC    2018-01-16  capricorn
AAL    2005-09-27      libra
...           ...        ...
ZTS    2013-02-01   aquarius
ZUMZ   2005-05-06     taurus
ZUO    2018-04-12      aries
ZYME   2017-04-28     taurus
ZYNE   2015-08-05        leo

[5685 rows x 2 columns]


In [9]:
stocks = stocks.merge(tickers[['SIGN']], on='TICKER', how='left')
stocks.head(3)

,TICKER,OPEN,CLOSE,ADJ_CLOSE,LOW,HIGH,VOLUME,DATE,SIGN
0,AHH,10.210000038147,9.88000011444092,7.40945959091187,9.84000015258789,10.2200002670288,53000,2013-12-02,taurus
1,AHH,9.84000015258789,10.0900001525879,7.56694889068604,9.65999984741211,10.210000038147,102900,2013-12-03,taurus
2,AHH,10.0799999237061,10.039999961853,7.52945232391357,9.89999961853027,10.2799997329712,80900,2013-12-04,taurus


In [40]:
merged = stocks.merge(horos[["DATE", "SIGN", "ROBERTA", "VADERS"]], on=['DATE', 'SIGN'], how='left')
merged.head(3)
merged = merged.drop(columns=['OPEN', 'CLOSE', 'LOW'])

In [41]:
merged = merged.sort_values(by=['TICKER', 'DATE'])
merged['ADJ_CLOSE'] = pd.to_numeric(merged['ADJ_CLOSE'], errors='coerce')
merged['HIGH'] = pd.to_numeric(merged['HIGH'], errors='coerce')
merged['VOLUME'] = pd.to_numeric(merged['VOLUME'], errors='coerce')

merged['ADJ_CLOSE_diff'] = merged.groupby('TICKER')['ADJ_CLOSE'].diff()
merged['HIGH_diff'] = merged.groupby('TICKER')['HIGH'].diff()
merged['VOLUME_diff'] = merged.groupby('TICKER')['VOLUME'].diff()
merged['ADJ_CLOSE_NORM'] = merged.groupby('TICKER').apply(lambda x: (x['ADJ_CLOSE_diff'] / x['ADJ_CLOSE'].shift(1)) * 100).reset_index(level=0, drop=True)
merged['HIGH_NORM'] = merged.groupby('TICKER').apply(lambda x: (x['HIGH_diff'] / x['HIGH'].shift(1)) * 100).reset_index(level=0, drop=True)
merged['VOLUME_NORM'] = merged.groupby('TICKER').apply(lambda x: (x['VOLUME_diff'] / x['VOLUME'].shift(1)) * 100).reset_index(level=0, drop=True)

merged

,TICKER,ADJ_CLOSE,HIGH,VOLUME,DATE,SIGN,ROBERTA,VADERS,ADJ_CLOSE_diff,HIGH_diff,VOLUME_diff,ADJ_CLOSE_NORM,HIGH_NORM,VOLUME_NORM
2304675,A,36.183960,38.390556,2039900,2013-12-02,scorpio,-0.7119106370955706,0.0387,NaN,NaN,NaN,NaN,NaN,NaN
2304676,A,35.993546,38.032906,3462700,2013-12-03,scorpio,-0.4503345638513565,0.5859,-0.190414,-0.357651,1422800.0,-0.526240,-0.931611,69.748517
2304680,A,36.435562,38.505009,3377200,2013-12-04,scorpio,0.3346603661775589,0.8979,0.442017,0.472103,-85500.0,1.228044,1.241302,-2.469171
2304681,A,36.381161,38.483547,2530900,2013-12-05,scorpio,0.35106323944768436,0.796,-0.054401,-0.021461,-846300.0,-0.149309,-0.055737,-25.059221
2304685,A,37.285587,39.291847,4268500,2013-12-06,scorpio,0.8115046685561538,0.2315,0.904427,0.808300,1737600.0,2.485975,2.100378,68.655419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834702,ZYNE,14.460000,14.640000,306100,2016-12-06,leo,-0.2466597145294973,0.7703,1.280000,0.640000,101100.0,9.711682,4.571431,49.317073
1834705,ZYNE,14.510000,14.660000,185900,2016-12-07,leo,-0.46127180896829423,0.3818,0.050000,0.020000,-120200.0,0.345783,0.136609,-39.268213
1834708,ZYNE,15.530000,15.720000,324900,2016-12-08,leo,-0.27889271825551987,0.1531,1.020000,1.060000,139000.0,7.029631,7.230562,74.771382
1834711,ZYNE,15.070000,16.590000,349300,2016-12-09,leo,0.3559838756918907,0.6577,-0.460000,0.870000,24400.0,-2.962009,5.534350,7.510003


In [42]:
merged.to_csv('db.csv', sep=';')